In [1]:
from query_precip import *
from query_temp import *
from query_rates import *
from query_helpers import *

import datetime
import time

from pycds import *
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
import pgpasslib
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
df_q = pd.read_sql(query_variables(session).statement, engine)
df_q.head()

,vars_id,standard_name,long_description,unit,cell_method,net_var_name
0,1401,,Hail,boolean,,16
1,1424,,Smoke or Haze,boolean,,18
2,1291,cloud_base_altitude,Ceiling currently observed,30s of meters,time: point,71
3,1292,visibility_in_air,Visibility currently observed,0.1 km,time: point,72
4,1293,air_pressure_at_sea_level,Sea level pressure,0.01 kPa,time: point,73


### PCICFlags table in dbmsc and crmp
dbmsc was contstructed using Python as opposed to crmp which was created using pure sql, nevetheless, we have options on how was want to deal with QC in the databases moving forward.

Example sql from crmp on PCIC flags:
```sql
 Table "crmp.meta_pcic_flag"
    Column    |          Type          | Collation | Nullable |                Default
--------------+------------------------+-----------+----------+---------------------------------------
 pcic_flag_id | integer                |           | not null | nextval('pcic_flag_id_seq'::regclass)
 flag_name    | character varying(255) |           |          |
 description  | character varying(255) |           |          |
 discard      | boolean                |           |          |
Indexes:
    "meta_pcic_flags_pkey" PRIMARY KEY, btree (pcic_flag_id)
Referenced by:
    TABLE "obs_raw_pcic_flags" CONSTRAINT "obs_raw_pcic_flags_pcic_flag_id_fkey" FOREIGN KEY (pcic_flag_id) REFERENCES meta_pcic_flag(pcic_flag_id)
```

In [ ]:
# Question for James:
# When we end up appending a flag to an obs object, what do we do with obs that require more than on PCICFlag?

# Can we add a flag to each observation of a given variable? Each variable will have different flags that wont 
# make sense to add to all of the variables, so we should select flags that only make sense for each respective variable and apply them?

# Examples:
# flag_name: exceed_world_record_rainfall
# description: Precipitation < 0 or > 1828.8
# discard: True

# flag_name: trace_value_precip
# description: larger than 0 and measurement flag = T(trace) (external)
# discard: True

# flag_name: duplicate_check_monthly
# description: All values in one month = all values in a nother month
# discard: True

# flag_name: duplicate_check_annually
# description: All values in one year = all corresponding values in another year
# discard: True

# flag_name: duplicate_check_same_month_not_years
# description: All values in one month = all values in another month (different)
# discard: True

# flag_name: world_record_precip
# description: Precipitation < 0 or > 1828.8
# discard: True 

# flag_name: identical_value
# description: indentical values 90 or more or whatever
# discard: True

# flag_name: identical_frequent_value_precip
# description: indentical values outside of the percentiles
# discard: True

In [ ]:
# This creates a Python object
a_flag = pycds.PCICFlag(
    flag_name='test_flag',
    description='test flag insertion into dbmsc',
    discard=False
)

# This adds it to a session
session.add(a_flag)
# At this point `a_flag` is an object that has a connection to the database, so you can add related objects
session.commit()

In [ ]:
for obs in list_of_obs_that_are_known_to_be_bad:
    obs.pcic_flags.append(a_flag)
session.commit()

In [ ]:
insert = PCICFlag.__table__.insert().values(
            flag_name='test_flag', 
            description='test flag insertion into dbmsc',
            discard=False
)

insert = PCICFlag.__table__.insert().values(
            flag_name='negative_rain', 
            description='you get the idea...',
            discard=True
)



session.execute(insert)